# Semantic Kernel

Sa code sample na ito, gagamitin mo ang [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework upang gumawa ng isang simpleng ahente.

Ang layunin ng sample na ito ay ipakita sa iyo ang mga hakbang na gagamitin natin sa mga susunod na code sample kapag ipinatutupad ang iba't ibang mga pattern ng ahente.


## I-import ang Mga Kailangan na Python Package


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Paglikha ng Client

Sa halimbawang ito, gagamitin natin ang [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para sa access sa LLM.

Ang `ai_model_id` ay tinukoy bilang `gpt-4o-mini`. Subukang palitan ang modelo sa ibang modelo na makukuha sa GitHub Models marketplace upang makita ang iba't ibang resulta.

Para magamit natin ang `Azure Inference SDK` na ginagamit para sa `base_url` ng GitHub Models, gagamitin natin ang `OpenAIChatCompletion` connector sa loob ng Semantic Kernel. Mayroon ding iba pang [mga magagamit na connector](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) upang magamit ang Semantic Kernel para sa iba pang mga provider ng modelo.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Paglikha ng Agent

Sa ibaba, gagawa tayo ng Agent na tinatawag na `TravelAgent`.

Para sa halimbawang ito, gumagamit tayo ng napakasimpleng mga tagubilin. Maaari mong baguhin ang mga tagubilin upang makita kung paano naiiba ang magiging tugon ng agent.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Pagpapatakbo ng Ahente

Ngayon, maaari na nating patakbuhin ang Ahente sa pamamagitan ng pagde-define ng thread na may uri na `ChatHistoryAgentThread`. Ang anumang kinakailangang mensahe ng sistema ay ibinibigay sa `messages` keyword argument ng invoke_stream ng ahente.

Kapag na-define na ang mga ito, gagawa tayo ng `user_inputs` na siyang ipapadala ng user sa ahente. Sa kasong ito, itinakda natin ang mensahe sa `Plan me a sunny vacation`.

Malaya kang baguhin ang mensaheng ito upang makita kung paano magre-react nang iba ang ahente.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
